# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.228261044029448)),
 (1, np.float64(1.228261044029448)),
 (2, np.float64(1.228261044029448)),
 (3, np.float64(1.228261044029448)),
 (4, np.float64(1.228261044029448))]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.02639236668671141)),
   (None, np.float64(0.16895440824643992)),
   (None, np.float64(0.17357878039462604)),
   (None, np.float64(0.20102078188930894)),
   (None, np.float64(0.22700470672946182)),
   (None, np.float64(0.2296179259386677)),
   (None, np.float64(0.2527086681425015)),
   (None, np.float64(0.2771221360005933)),
   (None, np.float64(0.2941101562327223)),
   (None, np.float64(0.3011895789236535)),
   (None, np.float64(0.312867198756102)),
   (None, np.float64(0.43587885962671646)),
   (None, np.float64(0.48257047740189074))]),
 (1,
  [(None, np.float64(0.5453325155382901)),
   (None, np.float64(0.5536265028260795)),
   (None, np.float64(0.5659313689324721)),
   (None, np.float64(0.6484760590692149)),
   (None, np.float64(0.6676222316576662)),
   (None, np.float64(0.7908648389861807)),
   (None, np.float64(0.8132814403670818)),
   (None, np.float64(0.8439631327583961)),
   (None, np.float64(0.8570089029027599)),
   (None, np.float64(0.8741458000085

In [21]:
# TeraSort: соберём итоговую отсортированную последовательность
# partitioned_output уже содержит отсортированные ключи внутри каждого reducer'а
sorted_values = [v for (_, part) in sorted(partitioned_output, key=lambda x: x[0]) for (_, v) in part]

sorted_values[:10], np.allclose(sorted_values, sorted(input_values)), len(sorted_values)


([np.float64(0.02639236668671141),
  np.float64(0.16895440824643992),
  np.float64(0.17357878039462604),
  np.float64(0.20102078188930894),
  np.float64(0.22700470672946182),
  np.float64(0.2296179259386677),
  np.float64(0.2527086681425015),
  np.float64(0.2771221360005933),
  np.float64(0.2941101562327223),
  np.float64(0.3011895789236535)],
 True,
 30)

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [22]:
# Максимальное значение ряда (MapReduce)
numbers = [7, 1, 10, -3, 10, 5]

def RECORDREADER():
    for idx, x in enumerate(numbers):
        yield (idx, x)

def MAP(_, x):
    yield ("max", x)

def REDUCE(k, values):
    yield (k, max(values))

mr_out = list(MapReduce(RECORDREADER, MAP, REDUCE))
mr_out, max(numbers)


([('max', 10)], 10)

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [23]:
# Арифметическое среднее (MapReduce)
numbers = [7, 1, 10, -3, 10, 5]

def RECORDREADER():
    for idx, x in enumerate(numbers):
        yield (idx, x)

def MAP(_, x):
    yield ("avg", (x, 1))

def REDUCE(k, pairs):
    s = 0.0
    n = 0
    for (x, c) in pairs:
        s += x
        n += c
    yield (k, s / n)

mr_avg = dict(MapReduce(RECORDREADER, MAP, REDUCE))["avg"]
mr_avg, (sum(numbers) / len(numbers))


(5.0, 5.0)

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [24]:
# GroupByKey на основе сортировки
from itertools import groupby

def groupbykey_sort(iterable):
    """Группировка (k,v) по ключу k через предварительную сортировку."""
    iterable = sorted(iterable, key=lambda kv: kv[0])
    for k, group in groupby(iterable, key=lambda kv: kv[0]):
        yield (k, [v for (_, v) in group])

example = [('b', 1), ('a', 2), ('b', 3), ('a', 4), ('c', 5)]
list(groupbykey(example)), list(groupbykey_sort(example))


([('b', [1, 3]), ('a', [2, 4]), ('c', [5])],
 [('a', [2, 4]), ('b', [1, 3]), ('c', [5])])

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
# Drop duplicates / distinct (MapReduce)
values = ["a", "b", "a", "c", "b", "d", "d", "e"]

def RECORDREADER():
    for idx, v in enumerate(values):
        yield (idx, v)

def MAP(_, v):
    yield (v, None)

def REDUCE(v, _):
    # один раз на ключ
    yield (v, None)

distinct = sorted([k for (k, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
distinct


['a', 'b', 'c', 'd', 'e']

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [26]:
# Selection (Выборка) (MapReduce)
# Отношение R: (name, age, gender)
R = [
    ("alice", 20, "F"),
    ("bob", 17, "M"),
    ("carol", 30, "F"),
    ("dave", 16, "M"),
    ("eva", 22, "F"),
]

def predicate(t):
    # пример предиката C: совершеннолетние
    return t[1] >= 18

def RECORDREADER():
    for idx, t in enumerate(R):
        yield (idx, t)

def MAP(_, t):
    if predicate(t):
        yield (t, t)

def REDUCE(t, _):
    yield (t, t)

selection = [t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
selection


[('alice', 20, 'F'), ('carol', 30, 'F'), ('eva', 22, 'F')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
# Projection (Проекция) (MapReduce)
# Проецируем отношение R на подмножество атрибутов S = {name, gender}
S_idx = (0, 2)  # индексы атрибутов

def RECORDREADER():
    for idx, t in enumerate(R):
        yield (idx, t)

def MAP(_, t):
    t_p = tuple(t[i] for i in S_idx)
    yield (t_p, t_p)

def REDUCE(t_p, _):
    yield (t_p, t_p)

projection = sorted([t_p for (t_p, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
projection


[('alice', 'F'), ('bob', 'M'), ('carol', 'F'), ('dave', 'M'), ('eva', 'F')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
# Union (Объединение) (MapReduce)
R1 = [(1, "a"), (2, "b"), (3, "c"), (3, "c")]  # есть дубликат внутри R1
S1 = [(2, "b"), (4, "d"), (4, "d")]            # есть дубликат внутри S1

def RECORDREADER():
    for idx, t in enumerate(R1):
        yield (("R", idx), t)
    for idx, t in enumerate(S1):
        yield (("S", idx), t)

def MAP(_, t):
    yield (t, t)

def REDUCE(t, _):
    yield (t, t)

union = sorted([t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
union


[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
# Intersection (Пересечение) (MapReduce)
def RECORDREADER():
    for idx, t in enumerate(R1):
        yield (("R", idx), ("R", t))
    for idx, t in enumerate(S1):
        yield (("S", idx), ("S", t))

def MAP(_, src_t):
    src, t = src_t
    yield (t, src)

def REDUCE(t, srcs):
    srcs = set(srcs)
    if ("R" in srcs) and ("S" in srcs):
        yield (t, t)

intersection = sorted([t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
intersection


[(2, 'b')]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
# Difference (Разница) R \ S (MapReduce)
def RECORDREADER():
    for idx, t in enumerate(R1):
        yield (("R", idx), ("R", t))
    for idx, t in enumerate(S1):
        yield (("S", idx), ("S", t))

def MAP(_, src_t):
    src, t = src_t
    yield (t, src)

def REDUCE(t, srcs):
    srcs = set(srcs)
    if srcs == {"R"}:
        yield (t, t)

difference = sorted([t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
difference


[(1, 'a'), (3, 'c')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
# Natural Join (Естественное соединение) (MapReduce)
# R(a,b), S(b,c)
R2 = [("x", 1), ("y", 2), ("z", 1)]
S2 = [(1, "A"), (1, "B"), (3, "C")]

def RECORDREADER():
    for idx, t in enumerate(R2):
        yield (("R", idx), ("R", t))
    for idx, t in enumerate(S2):
        yield (("S", idx), ("S", t))

def MAP(_, src_t):
    src, t = src_t
    if src == "R":
        a, b = t
        yield (b, ("R", a))
    else:
        b, c = t
        yield (b, ("S", c))

def REDUCE(b, tagged):
    left = [x[1] for x in tagged if x[0] == "R"]
    right = [x[1] for x in tagged if x[0] == "S"]
    for a in left:
        for c in right:
            yield ((a, b, c), None)

join_result = sorted([triple for (triple, _) in MapReduce(RECORDREADER, MAP, REDUCE)])
join_result


[('x', 1, 'A'), ('x', 1, 'B'), ('z', 1, 'A'), ('z', 1, 'B')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [32]:
# Grouping and Aggregation (Группировка и аггрегация) (MapReduce)
# Входные кортежи (a, b, c). Группируем по a и агрегируем b.
T = [("g1", 5, "x"), ("g1", 2, "y"), ("g2", 3, "z"), ("g2", 10, "w")]

def aggregate(theta):
    def RECORDREADER():
        for idx, t in enumerate(T):
            yield (idx, t)

    def MAP(_, t):
        a, b, _c = t
        yield (a, b)

    def REDUCE(a, bs):
        yield (a, theta(bs))

    return dict(MapReduce(RECORDREADER, MAP, REDUCE))

sum_by_group = aggregate(sum)
max_by_group = aggregate(max)

sum_by_group, max_by_group


({'g1': 7, 'g2': 13}, {'g1': 5, 'g2': 10})

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [33]:
# Matrix-Vector multiplication: вектор НЕ помещается в памяти Map-задачи
import numpy as np

I, J = 5, 4
M = np.random.randint(0, 5, size=(I, J)).astype(float)
v = np.random.randint(0, 5, size=(J,)).astype(float)

# 1) join по индексу j: (i,j,m_ij) natural join (j,v_j) -> (i, m_ij * v_j)
def RECORDREADER1():
    for i in range(I):
        for j in range(J):
            yield (("M", i, j), M[i, j])
    for j in range(J):
        yield (("V", j), v[j])

def MAP1(k1, val):
    tag = k1[0]
    if tag == "M":
        _, i, j = k1
        yield (j, ("M", i, val))
    else:
        _, j = k1
        yield (j, ("V", val))

def REDUCE1(j, tagged):
    vj = None
    m_items = []
    for item in tagged:
        if item[0] == "V":
            vj = item[1]
        else:
            _, i, mij = item
            m_items.append((i, mij))
    if vj is None:
        return
    for i, mij in m_items:
        yield (i, mij * vj)

stage1 = list(MapReduce(RECORDREADER1, MAP1, REDUCE1))

# 2) группировка по i и суммирование вкладов
def RECORDREADER2():
    for (i, prod) in stage1:
        yield (i, prod)

def MAP2(i, prod):
    yield (i, prod)

def REDUCE2(i, prods):
    yield (i, sum(prods))

y_dict = dict(MapReduce(RECORDREADER2, MAP2, REDUCE2))
y = np.array([y_dict.get(i, 0.0) for i in range(I)])

ref = M.dot(v)

M, v, y, ref, np.allclose(y, ref)


(array([[2., 2., 3., 3.],
        [3., 0., 3., 3.],
        [4., 3., 3., 2.],
        [3., 0., 0., 3.],
        [0., 3., 1., 4.]]),
 array([2., 1., 0., 3.]),
 array([15., 15., 17., 15., 15.]),
 array([15., 15., 17., 15., 15.]),
 True)

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [34]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [35]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # N(j,k) = w. Так как M (small_mat) в памяти, размножаем по i.
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))


Проверьте своё решение

In [36]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [37]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [38]:
# Matrix multiplication (обе матрицы генерируются в RECORDREADER)
import numpy as np

I, J, K = 3, 4, 5
A = np.random.rand(I, J)
B = np.random.rand(J, K)

def RECORDREADER():
    # Сначала A (M)
    for i in range(I):
        for j in range(J):
            yield (("M", i, j), A[i, j])
    # Затем B (N)
    for j in range(J):
        for k in range(K):
            yield (("N", j, k), B[j, k])

def MAP(k1, v1):
    tag = k1[0]
    if tag == "M":
        _, i, j = k1
        for k in range(K):
            yield ((i, k), ("M", j, v1))
    else:
        _, j, k = k1
        for i in range(I):
            yield ((i, k), ("N", j, v1))

def REDUCE(key, values):
    # key = (i,k)
    m = {}
    n = {}
    for (tag, j, val) in values:
        if tag == "M":
            m[j] = m.get(j, 0.0) + val
        else:
            n[j] = n.get(j, 0.0) + val
    s = 0.0
    for j in set(m.keys()).intersection(n.keys()):
        s += m[j] * n[j]
    yield (key, s)

out = list(MapReduce(RECORDREADER, MAP, REDUCE))

P = np.zeros((I, K))
for ((i, k), val) in out:
    P[i, k] = val

ref = A.dot(B)
np.allclose(P, ref), P, ref


(True,
 array([[0.53618552, 0.85334432, 0.94800648, 1.2734414 , 0.91920346],
        [0.74555687, 1.16289148, 0.59956967, 0.85174886, 1.19028946],
        [0.43497037, 0.69341022, 0.4901342 , 0.67984937, 0.66452777]]),
 array([[0.53618552, 0.85334432, 0.94800648, 1.2734414 , 0.91920346],
        [0.74555687, 1.16289148, 0.59956967, 0.85174886, 1.19028946],
        [0.43497037, 0.69341022, 0.4901342 , 0.67984937, 0.66452777]]))

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [39]:
# Matrix multiplication: MapReduce Distributed (каждая матрица в своём RECORDREADER)
import numpy as np

I, J, K = 4, 5, 6
A = np.random.rand(I, J)
B = np.random.rand(J, K)

reducers = 3  # важно для PARTITIONER из модельного кода

def INPUTFORMAT():
    def RR_A():
        for i in range(I):
            for j in range(J):
                yield (("M", i, j), A[i, j])

    def RR_B():
        for j in range(J):
            for k in range(K):
                yield (("N", j, k), B[j, k])

    yield RR_A()
    yield RR_B()

def MAP(k1, v1):
    tag = k1[0]
    if tag == "M":
        _, i, j = k1
        for k in range(K):
            yield ((i, k), ("M", j, v1))
    else:
        _, j, k = k1
        for i in range(I):
            yield ((i, k), ("N", j, v1))

def REDUCE(key, values):
    m = {}
    n = {}
    for (tag, j, val) in values:
        if tag == "M":
            m[j] = m.get(j, 0.0) + val
        else:
            n[j] = n.get(j, 0.0) + val
    s = 0.0
    for j in set(m.keys()).intersection(n.keys()):
        s += m[j] * n[j]
    yield (key, s)

reduce_partitions = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)

out = []
for (_, part) in reduce_partitions:
    out.extend(list(part))

P = np.zeros((I, K))
for ((i, k), val) in out:
    P[i, k] = val

ref = A.dot(B)
np.allclose(P, ref)


240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [40]:
# Обобщение: несколько RECORDREADER-ов на каждую матрицу + проверка случайных подмножеств
import numpy as np

I, J, K = 4, 5, 6
A = np.random.rand(I, J)
B = np.random.rand(J, K)

reducers = 4

def MAP(k1, v1):
    tag = k1[0]
    if tag == "M":
        _, i, j = k1
        for k in range(K):
            yield ((i, k), ("M", j, v1))
    else:
        _, j, k = k1
        for i in range(I):
            yield ((i, k), ("N", j, v1))

def REDUCE(key, values):
    m = {}
    n = {}
    for (tag, j, val) in values:
        if tag == "M":
            m[j] = m.get(j, 0.0) + val
        else:
            n[j] = n.get(j, 0.0) + val
    s = 0.0
    for j in set(m.keys()).intersection(n.keys()):
        s += m[j] * n[j]
    yield (key, s)

# --- 1) Корректное разбиение на несколько RECORDREADER-ов (все элементы присутствуют ровно один раз)
def rr_A(i_start, i_end):
    for i in range(i_start, i_end):
        for j in range(J):
            yield (("M", i, j), A[i, j])

def rr_B(k_start, k_end):
    for j in range(J):
        for k in range(k_start, k_end):
            yield (("N", j, k), B[j, k])

def INPUTFORMAT_FULL():
    # A: 2 разбиения по строкам
    mid = I // 2
    yield rr_A(0, mid)
    yield rr_A(mid, I)

    # B: 3 разбиения по столбцам (по k)
    cuts = np.linspace(0, K, 4, dtype=int)
    for ks, ke in zip(cuts[:-1], cuts[1:]):
        yield rr_B(int(ks), int(ke))

out_full = []
for (_, part) in MapReduceDistributed(INPUTFORMAT_FULL, MAP, REDUCE):
    out_full.extend(list(part))

P_full = np.zeros((I, K))
for ((i, k), val) in out_full:
    P_full[i, k] = val

ref = A.dot(B)
ok_full = np.allclose(P_full, ref)

# --- 2) Случайные подмножества элементов (пропуски) => корректно только если пропущенные элементы считаются нулями
rng = np.random.default_rng(0)
maskA = (rng.random((I, J)) < 0.7)
maskB = (rng.random((J, K)) < 0.7)

def rr_A_subset():
    for i in range(I):
        for j in range(J):
            if maskA[i, j]:
                yield (("M", i, j), A[i, j])

def rr_B_subset():
    for j in range(J):
        for k in range(K):
            if maskB[j, k]:
                yield (("N", j, k), B[j, k])

def INPUTFORMAT_SUBSET():
    yield rr_A_subset()
    yield rr_B_subset()

out_sub = []
for (_, part) in MapReduceDistributed(INPUTFORMAT_SUBSET, MAP, REDUCE):
    out_sub.extend(list(part))

P_sub = np.zeros((I, K))
for ((i, k), val) in out_sub:
    P_sub[i, k] = val

ref_dense = ref
ref_sparse = (A * maskA.astype(float)).dot(B * maskB.astype(float))

(
    ok_full,
    np.allclose(P_sub, ref_dense),
    np.allclose(P_sub, ref_sparse),
    "Случайные подмножества корректны только при трактовке пропусков как нулей (разреженное хранение)."
)


240 key-value pairs were sent over a network.
164 key-value pairs were sent over a network.


(True,
 False,
 True,
 'Случайные подмножества корректны только при трактовке пропусков как нулей (разреженное хранение).')